In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#from google.colab import drive

In [2]:
#drive.mount('/content/drive') # Uncomment if loading from Google Drive
#cancer_df = pd.read_csv('/content/drive/My Drive/PROYECTOS/DM_II_project/data/raw/patient_train_data.csv') # Uncomment if loading from Google Drive .Load the data from Google Drive
cancer_df = pd.read_csv('g:/Mi unidad/PROYECTOS/DM_II_project/data/raw/patient_train_data.csv') # Uncomment if loading from local directory

In [3]:
cancer_df.head()

,ID,Alcohol Consumption,Cancer Stage,Country,Date of Birth,Diabetes,Diabetes History,Diet Risk,Early Detection,Family History,...,Non Smoker,Obesity BMI,Physical Activity,Screening History,Smoking History,Transfusion History,Treatment Type,Tumor Size (mm),Urban or Rural,Survival Prediction
0,1,No,Localized,UK,29-01-1966,No,No,Moderate,No,No,...,Yes,Overweight,Low,Regular,No,-,Chemotherapy,33.0,Urban,Yes
1,2,No,Regional,Japan,21-12-1958,No,No,Low,No,No,...,No,Normal,Low,Irregular,Yes,-,Chemotherapy,17.0,Urban,No
2,3,No,Localized,France,16-06-1959,No,No,Low,Yes,No,...,No,Normal,Moderate,Never,Yes,-,Surgery,34.0,Urban,Yes
3,4,Yes,Localized,Japan,18-01-1956,Yes,No,Moderate,No,No,...,Yes,Obese,Low,Regular,No,-,Radiotherapy,71.0,Urban,No
4,5,No,Regional,USA,04-05-1975,No,No,Moderate,Yes,Yes,...,Yes,Overweight,Low,Regular,No,-,Radiotherapy,48.0,Rural,Yes


# 1. Basic cleaning and data split

Three columns were dropped from the `cancer_df` dataframe: 'Diabetes History', 'Marital Status', and 'Transfusion History'.

In [33]:
cancer_df_reduced = cancer_df.drop(columns=['Diabetes History', 'Marital Status', 'Transfusion History'], axis=1).copy()

The country names in the `cancer_df_reduced` dataframe were replaced by their respective continents to reduce the cardinality of the categorical variable. This helps in simplifying the analysis and modeling by grouping countries into broader regions.

In [34]:
cancer_df_reduced['Country'].unique()

array(['UK', 'Japan', 'France', 'USA', 'China', 'South Korea', 'Brazil',
       'Germany', 'Canada', 'Pakistan', 'Italy', 'New Zealand',
       'South Africa', 'India', 'Nigeria', 'Australia', nan], dtype=object)

In [35]:
continents = {
    'USA': 'North America',
    'Brazil': 'South America',
    'Germany': 'Europe',
    'France': 'Europe',
    'Italy': 'Europe',
    'UK': 'Europe',
    'China': 'Asia',
    'Japan': 'Asia',
    'India': 'Asia',
    'South Korea': 'Asia',
    'Pakistan': 'Asia',
    'South Africa': 'Africa',
    'Nigeria': 'Africa',
    'Australia': 'Oceania',
    'New Zealand': 'Oceania'
}
cancer_df_reduced['Continent'] = cancer_df_reduced['Country'].replace(continents)
cancer_df_reduced.drop(columns=['Country'], inplace=True)
cancer_df_reduced['Continent'].head()

0           Europe
1             Asia
2           Europe
3             Asia
4    North America
Name: Continent, dtype: object

The birth dates in the dataframe were converted to datetime format, and the age of each patient was calculated based on the current year. The original birth date column was then removed from the dataframe.

In [36]:
from datetime import datetime

cancer_df_reduced['Date of Birth'] = pd.to_datetime(cancer_df_reduced['Date of Birth'], format='%d-%m-%Y', errors='coerce')
cancer_df_reduced['Age'] = cancer_df_reduced['Date of Birth'].apply(lambda x: datetime.now().year - x.year if pd.notnull(x) else None)
cancer_df_reduced.drop(columns=['Date of Birth'], inplace=True)
cancer_df_reduced['Age'].head()

0    59.0
1    67.0
2    66.0
3    69.0
4    50.0
Name: Age, dtype: float64

The 'Urban or Rural' column values were standardized by replacing 'urban' with 'Urban' and 'rural' with 'Rural' to ensure consistency in the dataset.

In [37]:
cancer_df_reduced['Urban or Rural'].unique()

array(['Urban', 'Rural', 'urban', 'rural', nan], dtype=object)

In [38]:
cancer_df_reduced['Urban or Rural'].replace({'urban': 'Urban', 'rural': 'Rural'}, inplace=True)
cancer_df_reduced['Urban or Rural'].unique()

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_15828\4237172287.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cancer_df_reduced['Urban or Rural'].replace({'urban': 'Urban', 'rural': 'Rural'}, inplace=True)


array(['Urban', 'Rural', nan], dtype=object)

In [39]:
cancer_df_reduced.head()

,ID,Alcohol Consumption,Cancer Stage,Diabetes,Diet Risk,Early Detection,Family History,Gender,Genetic Mutation,Healthcare Access,...,Obesity BMI,Physical Activity,Screening History,Smoking History,Treatment Type,Tumor Size (mm),Urban or Rural,Survival Prediction,Continent,Age
0,1,No,Localized,No,Moderate,No,No,M,No,High,...,Overweight,Low,Regular,No,Chemotherapy,33.0,Urban,Yes,Europe,59.0
1,2,No,Regional,No,Low,No,No,M,No,Moderate,...,Normal,Low,Irregular,Yes,Chemotherapy,17.0,Urban,No,Asia,67.0
2,3,No,Localized,No,Low,Yes,No,M,No,High,...,Normal,Moderate,Never,Yes,Surgery,34.0,Urban,Yes,Europe,66.0
3,4,Yes,Localized,Yes,Moderate,No,No,M,No,Moderate,...,Obese,Low,Regular,No,Radiotherapy,71.0,Urban,No,Asia,69.0
4,5,No,Regional,No,Moderate,Yes,Yes,M,No,Low,...,Overweight,Low,Regular,No,Radiotherapy,48.0,Rural,Yes,North America,50.0


The dataset was split into training, validation, and test sets to ensure robust model evaluation and to prevent overfitting. The initial split separated 15% of the data as the test set, which will be used for the final evaluation of the model's performance. This ensures that the test set remains completely unseen during the training and validation phases, providing an unbiased assessment of the model's generalization capability.

The proportions were chosen to maintain a balance between having enough data for training the model and having sufficient data for validation and testing to ensure reliable performance metrics.

In [40]:
from sklearn.model_selection import train_test_split

X = cancer_df_reduced.drop(columns=['Survival Prediction'])
y = cancer_df_reduced['Survival Prediction'] 

# First split to separate the test set (15% of the data)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# Split the remaining data into training and validation sets
# test_size=0.1765 because 15/85 ≈ 0.1765 (to get 15% of the original total)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, random_state=42, stratify=y_temp)

# Verify the sizes and proportions
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (52522, 28)
X_val shape: (11257, 28)
X_test shape: (11256, 28)
y_train shape: (52522,)
y_val shape: (11257,)
y_test shape: (11256,)
